In [13]:
import umap
import pandas as pd
import os

raw_dir = './raws'
ex_dir = './processed'

train = pd.read_csv(os.path.join(raw_dir, 'train.csv'))
greeks = pd.read_csv(os.path.join(raw_dir, 'greeks.csv'))

train.replace(['A', 'B'], [0, 1], inplace=True)
train

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,0.173229,1.26092,0.067730,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,10.223150,1.24236,0.426699,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,0.173229,0.49706,0.067730,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,9.256996,0.78764,0.670527,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0


In [38]:
# 次元削減
import umap

rand = 0
nc = 19
nn = 10

um = umap.UMAP(n_components=nc, n_neighbors=nn, random_state=rand)
x = train.drop(columns=['Id', 'Class'])
train.reset_index()

um.fit(x)
x_rd = pd.DataFrame(um.transform(x)).reset_index()
x_rd.columns = ['index'] + ['f'+str(i) for i in range(len(x_rd.columns)-1)]

# greeks.csvからのクラス抽出
x_rd = pd.concat([train.loc[:, ['Id', 'Class']], x_rd], axis=1)
x_rd.drop(columns='index', inplace=True)

x_rd = x_rd.merge(greeks.loc[:, 'Id':'Alpha'], on='Id')
l = x_rd['Alpha'].tolist()

x_rd['B'] = [1 if i=='B' else 0 for i in l]
x_rd['C'] = [1 if i=='C' else 0 for i in l]
x_rd['D'] = [1 if i=='D' else 0 for i in l]
x_rd.drop(columns='Alpha', inplace=True)

# 保存
x_rd.to_csv(os.path.join(ex_dir, 'train_rd.csv'), index=False)
x_rd

,Id,Class,f0,f1,f2,f3,f4,f5,f6,f7,...,f12,f13,f14,f15,f16,f17,f18,B,C,D
0,000ff2bfdfe9,1,10.993897,6.120290,6.710696,6.556782,5.521480,6.343551,5.129878,5.222734,...,5.697223,3.885029,5.470263,4.041500,5.821287,5.043557,6.790655,1,0,0
1,007255e47698,0,3.589066,9.164880,8.812347,4.377454,5.895328,4.586554,4.388479,4.993362,...,4.303647,4.368522,5.678960,5.342089,5.551815,4.403145,6.341142,0,0,0
2,013f2bd269f5,0,8.188018,8.465458,8.569057,4.889384,6.433257,5.132563,4.542758,4.844675,...,5.096869,3.413509,5.834739,5.146508,5.754706,4.678692,6.142032,0,0,0
3,043ac50845d5,0,10.168103,7.241946,7.791352,6.011106,6.085346,5.793283,4.716759,4.887436,...,5.258366,3.164863,5.533222,4.849071,5.612569,4.971673,6.060015,0,0,0
4,044fb8a146ec,1,8.591958,6.781187,6.548851,6.470881,5.552547,6.527836,5.141641,4.978046,...,5.445625,3.939907,5.218297,4.121197,6.127697,4.945318,6.518445,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0,8.383711,6.925010,6.540789,6.404557,5.616021,6.534841,5.141513,4.946859,...,5.416947,3.891917,5.208506,4.160072,6.149751,4.922191,6.483093,0,0,0
613,fd895603f071,0,8.682390,6.512975,6.216189,6.716962,5.476690,6.736801,5.256656,5.021194,...,5.511580,3.986274,5.162032,3.949647,6.170870,4.994534,6.585639,0,0,0
614,fd8ef6377f76,0,11.243439,6.861791,8.454022,5.559681,5.927985,4.938860,4.608397,5.450384,...,5.317402,3.603866,5.849700,4.867034,5.285856,4.885365,6.415137,0,0,0
615,fe1942975e40,0,12.171227,5.857488,6.931715,6.388868,5.671971,6.142674,5.134499,5.304612,...,5.719112,3.963717,5.763036,4.162627,5.592016,5.164902,6.857567,0,0,0


In [37]:
def func(df):
    # greeks.csvからのクラス抽出
    df.drop(columns='index', inplace=True, errors='ignore')

    df = df.merge(greeks.loc[:, 'Id':'Alpha'], on='Id')
    l = df['Alpha'].tolist()

    df['B'] = [1 if i=='B' else 0 for i in l]
    df['C'] = [1 if i=='C' else 0 for i in l]
    df['D'] = [1 if i=='D' else 0 for i in l]
    df.drop(columns='Alpha', inplace=True)
    return df

train.reset_index()
tr = func(train)
tr.to_csv(os.path.join(ex_dir, 'train.csv'), index=False)
tr

,Id,AB,AF,AH,AM,AR,AX,AY,AZ,BC,...,GB,GE,GF,GH,GI,GL,Class,B,C,D
0,000ff2bfdfe9,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,...,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1,1,0,0
1,007255e47698,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,...,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0,0,0,0
2,013f2bd269f5,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,...,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0,0,0,0
3,043ac50845d5,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,...,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0,0,0,0
4,044fb8a146ec,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,...,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
612,fd3dafe738fd,0.149555,3130.05946,123.763599,9.513984,13.020852,3.499305,0.077343,8.545512,2.804172,...,8.967128,217.148554,8095.932828,24.640462,69.191944,21.978000,0,0,0,0
613,fd895603f071,0.435846,5462.03438,85.200147,46.551007,15.973224,5.979825,0.025882,12.622906,3.777550,...,35.896418,496.994214,3085.308063,29.648928,124.808872,0.145340,0,0,0,0
614,fd8ef6377f76,0.427300,2459.10720,130.138587,55.355778,10.005552,8.070549,0.025578,15.408390,1.229900,...,19.962092,128.896894,6474.652866,26.166072,119.559420,21.978000,0,0,0,0
615,fe1942975e40,0.363205,1263.53524,85.200147,23.685856,8.138688,7.981959,0.025578,7.524588,1.229900,...,24.594488,72.611063,1965.343176,25.116750,37.155112,0.184622,0,0,0,0


In [17]:
def f(df):
    tmp = train.drop(columns=['Id', 'Class'])
    return (df - tmp.mean())/tmp.std()

train_ = f(train)
train_['Id'] = train['Id']
train_['Class'] = train['Class']

train_.fillna(train_.loc[:, 'AB':'GL'].median(), inplace=True)
train_.to_csv(os.path.join(ex_dir, 'train_st.csv'))